This function computes the (linear) interpolant for evaluating the differential equation:

\begin{equation}
\dfrac{d \phi}{ds} = - \dfrac{\cos^2(\phi)[C_x^{11}(\mathbf{x}_0)\cos(\phi)+C_y^{11}(\mathbf{x}_0)\sin(\phi)]+[C_x^{12}(\mathbf{x}_0)\cos(\phi)+C_y^{11}(\mathbf{x}_0)\sin(\phi)]\sin(2\phi)+\sin^2(\phi)[C_x^{22}(\mathbf{x}_0)\cos(\phi)+C_y^{22}(\mathbf{x}_0)\sin(\phi)]}{\sin(2\phi)[C^{22}(\mathbf{x}_0)-C^{11}(\mathbf{x}_0)]+2\cos(2\phi)C^{12}(\mathbf{x}_0)}
\label{eq: phi_prime} \tag{1}
\end{equation}

at every point $ (\mathbf{x}, \phi) $.

To do so, we first need to evaluate the right-hand-side of eq. \ref{eq: phi_prime} over a rectangular meshgrid $ X, Y, \phi $ (see *__phi_prime*) before computing the interpolant of $ \dot{\phi} $ (see *_phi_prime*).

In [1]:
# Import numpy
import numpy as np

# Import math symbols
from math import cos, sin, pi

# Import numba (used for high performance scientific computing)
from numba import jit, prange

In [1]:
@jit(parallel=True)
def __phi_prime(C11, C12, C22, C11_x, C11_y, C12_x, C12_y, C22_x, C22_y):
    '''
    Computes phi_prime and phi_prime_denom (denominator of phi prime) over gridded domain
    based on the elements of the Cauchy Green strain tensor
    
    Parameters:
        Cij:             array(Ny, Nx), ij-element of the Cauchy Green strain tensor over the gridded domain 
        Cij_x:           array(Ny, Nx), derivative with respect to x-th coordinate of the ij-element of the Cauchy Green strain tensor over the gridded domain 
        Cij_y:           array(Ny, Nx), derivative with respect to y-th coordinate of the ij-element of the Cauchy Green strain tensor over the gridded domain 
    
    Returns:
        phi:             array(360, ), angles
        phi_prime:       array(Ny, Nx, 360), contains phi_prime over gridded domain and angles
        phi_prime_denom: array(Ny, Nx, 360), contains denominator of phi_prime over gridded domain and angles. This specifies the domain of existence.
    '''
    
    # angles
    phi = np.linspace(0, 2*pi, 360)
    
    # define arrays
    phi_prime = np.zeros((C11.shape[0], C11.shape[1], phi.shape[0]))
    phi_prime_denom = phi_prime.copy()
    
    # iterate over meshgrid in y-direction
    for i in prange(C11.shape[0]):
        
        # iterate over meshgrid in x-direction
        for j in prange(C11.shape[1]):
            
            # elements of rate of strain
            C11_ = C11[i, j]
            C12_ = C12[i, j]
            C22_ = C22[i, j]
            
            # spatial derivative of elements of rate of strain
            C11_x_ = C11_x[i, j]
            C11_y_ = C11_y[i, j]
            
            C12_x_ = C12_x[i, j]
            C12_y_ = C12_y[i, j]
            
            C22_x_ = C22_x[i, j]
            C22_y_ = C22_y[i, j]

            # iterate over phi
            for p in prange(phi.shape[0]):
                
                phi_ = phi[p]
                
                # [C11_x*cos(phi_)+C11_y*sin(phi)]*cos^2(phi)
                phi_prime_num = (C11_x_*cos(phi_)+C11_y_*sin(phi_))*cos(phi_)**2
                
                # [C12_x*cos(phi_)+C12_y*sin(phi)]*sin(2*phi)
                phi_prime_num += (C12_x_*cos(phi_)+C12_y_*sin(phi_))*sin(2*phi_)
                
                # [C11_x*cos(phi_)+C11_y*sin(phi)]*sin^2(phi)
                phi_prime_num += (C22_x_*cos(phi_)+C22_y_*sin(phi_))*sin(phi_)**2
    
                # sin(2*phi)*(C22-C11)+2*cos(2*phi)*C12
                phi_prime_denom[i, j, p] = sin(2*phi_)*(C22_-C11_)+2*cos(2*phi_)*C12_
            
                phi_prime[i, j, p] = -phi_prime_num/phi_prime_denom[i, j, p]

    return phi_prime, phi_prime_denom, phi

In [2]:
def _phi_prime(X, Y, C11, C12, C22, C11_x, C11_y, C12_x, C12_y, C22_x, C22_y):
    
    '''
    Computes the linear interpolant of phi_prime and phi_prime_denom (denominator of phi prime) 
    based on the elements of the Cauchy Green strain tensor
    
    Parameters:
        X:                           array(Ny, Nx), X-meshgrid
        Y:                           array(Ny, Nx), Y-meshgrid
        Cij:                         array(Ny, Nx), ij-element of the Cauchy Green strain tensor over the gridded domain 
        Cij_x:                       array(Ny, Nx), derivative with respect to x-th coordinate of the ij-element of the Cauchy Green strain tensor over the gridded domain 
        Cij_y:                       array(Ny, Nx), derivative with respect to y-th coordinate of the ij-element of the Cauchy Green strain tensor over the gridded domain 
    
    Returns:
        Interpolant_phi_prime:       Interpolant-object for phi_prime over gridded domain and angles.
        Interpolant_phi_prime_denom: Interpolant-object for phi_prime_denom over gridded domain and angles.
    '''
    
    # Import interpolator from scipy
    from scipy.interpolate import RegularGridInterpolator
    
    # phi_prime and phi_prime_denom (=DOE) are arrays three dimensional arrays
    phi_prime, phi_prime_denom, phi = __phi_prime(C11, C12, C22, C11_x, C11_y, C12_x, C12_y, C22_x, C22_y)
    
    # Interpolant objects
    Interpolant_phi_prime = RegularGridInterpolator((Y[:,0], X[0,:], phi), phi_prime, bounds_error = False, fill_value = 0)
    Interpolant_phi_prime_denom = RegularGridInterpolator((Y[:,0], X[0,:], phi), phi_prime_denom)
    
    # return interpolant for phi_prime and for the denominator of phi_prime (=DOE)
    return Interpolant_phi_prime, Interpolant_phi_prime_denom